In [1]:
from census import Census
import censusdata
from us import states
import us
import requests
import zipfile
import io
import geopandas as gpd
import pandas as pd
import os
import urllib.request
import json
import requests,pandas as pd

In [2]:
# This is the key you can get from the census site
key = '5984786462aaccc39f904bf750cc8274bd40f9f8'

In [3]:
json_url = 'https://api.census.gov/data/2010/dec/sf1/variables.json'

with urllib.request.urlopen(json_url) as url:
    s = url.read()
    data = json.loads(s)


In [4]:
# These are the avaulable vaiables for the census 2010
data['variables']

{'for': {'label': "Census API FIPS 'for' clause",
  'concept': 'Census API Geography Specification',
  'predicateType': 'fips-for',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True},
 'in': {'label': "Census API FIPS 'in' clause",
  'concept': 'Census API Geography Specification',
  'predicateType': 'fips-in',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True},
 'ucgid': {'label': 'Uniform Census Geography Identifier clause',
  'concept': 'Census API Geography Specification',
  'predicateType': 'ucgid',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True},
 'P029009': {'label': 'Total!!In households!!In family households!!Adopted child',
  'concept': 'HOUSEHOLD TYPE BY RELATIONSHIP',
  'predicateType': 'int',
  'group': 'P29',
  'limit': 0},
 'P029007': {'label': 'Total!!In households!!In family households!!Spouse',
  'concept': 'HOUSEHOLD TYPE BY RELATIONSHIP',
  'predicateType': 'int',
  'group': 'P29',
  'limit': 0},
 'P029008': {'label': 'Total!!In households!!In f

In [5]:
# Variables of interest. Since API calls have a limit on the number of variables, I split the variables into two
race_and_pop = {'H010001': "Total population in occupied housing units",
             'P003002': "Total White alone",
             'P003003':'Total Black or African American alone',
             'P003004': 'Total American Indian and Alaska Native alone',
             'P003005': 'Total Asian alone',
              'P003006':'Total Native Hawaiian and Other Pacific Islander alone',
             'P003007': 'Total Some Other Race alone',
             'P003008': 'Two or More Races'}
sex_and_age = {'P012002':"Male",
             'P012026':"Female",
               'P012003':'Male Under 5 years',
             'P012004':'Male 5 to 9 years',
             'P012005': 'Male 10 to 14 years',
             'P012006': 'Male 15 to 17 years',
             'P012007':'Male 18 and 19 years',
             'P012008': 'Male 20 years',
             'P012009':'Male 21 years',
             'P012010': 'Male 22 to 24 years',
             'P012011': 'Male 25 to 29 year',
             'P012012':'Male 30 to 34 years',
             'P012013': 'Male 35 to 39 years',
             'P012014': 'Male 40 to 44 years',
             'P012015':'Male 45 to 49 years',
             'P012016':'Male 50 to 54 years',
             'P012017':'Male 55 to 59 years',
             'P012018': 'Male 60 and 61 years',
             'P012019':'Male 62 to 64 years',
             'P012020':'Male 65 and 66 years',
             'P012021':'Male 67 to 69 years',
             'P012022':'Male 70 to 74 years',
             'P012023':'Male 75 to 79 years',
             'P012024':'Male 80 to 84 years',
             'P012025': 'Male 85 years and over',
             'P012027':'Female Under 5 years',
             'P012028': 'Female 5 to 9 years',
             'P012029': 'Female 10 to 14 years',
             'P012030':'Female 15 to 17 years',
             'P012031':'Female 18 and 19 years',
             'P012032':'Female 20 years',
             'P012033':'Female 21 years',
             'P012034':'Female 22 to 24 years',
             'P012035':'Female 25 to 29 years',
             'P012036':'Female 30 to 34 years',
             'P012037':'Female 35 to 39 years',
             'P012039':'Female 45 to 49 years',
             'P012040':'Female 50 to 54 years',
             'P012041':'Female 55 to 59 years',
             'P012042': 'Female 60 and 61 years',
             'P012043':'Female 62 to 64 years',
             'P012044':'Female 65 and 66 years',
              'P012045':'Female 67 to 69 years',
             'P012046':'Female 70 to 74 years',
             'P012047':'Female 75 to 79 years',
             'P012048':'Female 80 to 84 years',
             'P012049':'Female 85 years and over'}

In [6]:
ordered_columns = ['NAME', 'state','county', 'tract', 'block', 'GEOID10','Total population in occupied housing units', 'Total White alone',
       'Total Black or African American alone','Total American Indian and Alaska Native alone', 'Total Asian alone',
       'Total Native Hawaiian and Other Pacific Islander alone','Total Some Other Race alone', 'Two or More Races',
            'Male', 'Female','Male Under 5 years', 'Male 5 to 9 years', 'Male 10 to 14 years',
       'Male 15 to 17 years', 'Male 18 and 19 years', 'Male 20 years','Male 21 years', 'Male 22 to 24 years', 'Male 25 to 29 year',
       'Male 30 to 34 years', 'Male 35 to 39 years', 'Male 40 to 44 years','Male 45 to 49 years', 'Male 50 to 54 years', 'Male 55 to 59 years',
       'Male 60 and 61 years', 'Male 62 to 64 years', 'Male 65 and 66 years','Male 67 to 69 years', 'Male 70 to 74 years', 'Male 75 to 79 years',
       'Male 80 to 84 years', 'Male 85 years and over', 'Female Under 5 years','Female 5 to 9 years', 'Female 10 to 14 years', 'Female 15 to 17 years',
       'Female 18 and 19 years', 'Female 20 years', 'Female 21 years','Female 22 to 24 years', 'Female 25 to 29 years',
       'Female 30 to 34 years', 'Female 35 to 39 years','Female 45 to 49 years', 'Female 50 to 54 years',
       'Female 55 to 59 years', 'Female 60 and 61 years','Female 62 to 64 years', 'Female 65 and 66 years',
       'Female 67 to 69 years', 'Female 70 to 74 years','Female 75 to 79 years', 'Female 80 to 84 years','Female 85 years and over']

In [8]:
dict_of_states = {'AK':['02','ALASKA'], 'MS':['28','MISSISSIPPI'],'AL':['01','ALABAMA'], 'MT':['30','MONTANA'],
'AR':['05','ARKANSAS'],'NC':['37','NORTH CAROLINA'], 'AS':['60','AMERICAN SAMOA'],
 'ND':['38','NORTH DAKOTA'], 'AZ':['04','ARIZONA'], 'NE': ['31','NEBRASKA'],
'CA':['06','CALIFORNIA'], 'NH':['33','NEW HAMPSHIRE'], 'CO':['08', 'COLORADO'], 'NJ':['34', 'NEW JERSEY'],
'CT':['09', 'CONNECTICUT'], 'NM':['35', 'NEW MEXICO'],'DC':['11','DISTRICT OF COLUMBIA'], 'NV':['32','NEVADA'],
'DE':['10', 'DELAWARE'], 'NY':['36','NEW YORK'], 'FL':['12','FLORIDA'], 'OH':['39','OHIO'],
'GA':['13','GEORGIA'], 'OK':['40','OKLAHOMA'], 'GU':['66','GUAM'], 'OR':['41','OREGON'],
'HI':['15','HAWAII'],'PA':['42','PENNSYLVANIA'],'IA':['19','IOWA'], 'PR':['72','PUERTO RICO'],
'ID':['16','IDAHO'],'RI':['44','RHODE ISLAND'],'IL':['17','ILLINOIS'],'SC':['45','SOUTH CAROLINA'],
'IN':['18','INDIANA'],'SD':['46','SOUTH DAKOTA'],'KS':['20','KANSAS'],'TN':['47','TENNESSEE'],
'KY':['21','KENTUCKY'],'TX':['48','TEXAS'],'LA':['22','LOUISIANA'],'UT':['49','UTAH'],'MA':['25','MASSACHUSETTS'],
'VA':['51','VIRGINIA'],'MD':['24','MARYLAND'],'VI':['78','VIRGIN ISLANDS'],'ME':['23','MAINE'],
'VT':['50','VERMONT'],'MI':['26','MICHIGAN'],'WA':['53','WASHINGTON'],'MN':['27','MINNESOTA'],'WI':['55','WISCONSIN'],
'MO':['29','MISSOURI'],'WV':['54','WEST VIRGINIA'],'WY': ['56','WYOMING']}

In [9]:
## I use the censusdata library to get the county names and state codes
censusdata.geographies(censusdata.censusgeo([('state', '*'), ('county', '*')]), 'sf1', 2010)

{'Sebastian County, Arkansas': censusgeo((('state', '05'), ('county', '131'))),
 'Sevier County, Arkansas': censusgeo((('state', '05'), ('county', '133'))),
 'Sharp County, Arkansas': censusgeo((('state', '05'), ('county', '135'))),
 'Stone County, Arkansas': censusgeo((('state', '05'), ('county', '137'))),
 'Union County, Arkansas': censusgeo((('state', '05'), ('county', '139'))),
 'Van Buren County, Arkansas': censusgeo((('state', '05'), ('county', '141'))),
 'Washington County, Arkansas': censusgeo((('state', '05'), ('county', '143'))),
 'White County, Arkansas': censusgeo((('state', '05'), ('county', '145'))),
 'Yell County, Arkansas': censusgeo((('state', '05'), ('county', '149'))),
 'Colusa County, California': censusgeo((('state', '06'), ('county', '011'))),
 'Butte County, California': censusgeo((('state', '06'), ('county', '007'))),
 'Alameda County, California': censusgeo((('state', '06'), ('county', '001'))),
 'Alpine County, California': censusgeo((('state', '06'), ('county

In [10]:
# This dictionary has the county codes and names for all states
d1 = censusdata.geographies(censusdata.censusgeo([('state', '*'), ('county', '*')]), 'sf1', 2010)

In [11]:
# This dictionary has the state codes

d2 = censusdata.geographies(censusdata.censusgeo([('state', '*')
                                                 ]), 'sf1', 2010)

In [12]:
# I create a couple of dictionaries to code the names and folders
dict_of_CENSUS_state_codes={}
dict_of_STATE_COUNTIES = {}
for key, val in d2.items():
    ls = []
    if key not in dict_of_CENSUS_state_codes:
        dict_of_CENSUS_state_codes[key] = list(d2[key].request().values())[0][-2:]
        for k, v in d1.items():
            if v.request()['in'][-2:] == dict_of_CENSUS_state_codes[key]:
                ls.append(v.request()['for'][-3:])
        dict_of_STATE_COUNTIES[key] = val.request()['for'][-2:],ls

In [13]:
def download_shp_and_save():
    '''
    Creates folders for each state, downloads the shapefile, unzip it and saves it into a folder
    for each state
    '''
    if not os.path.exists('shapefiles/'):
            os.mkdir('shapefiles/')
    for state, state_info in dict_of_states.items():
        if not os.path.exists('shapefiles/' + state_info[1]):
            os.mkdir('shapefiles/' + state_info[1])
        link = 'https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_{}_tabblock10.zip'.format(state_info[0])
        print(state_info[1])
        print(link)
        req = requests.get(link)
        file = zipfile.ZipFile(io.BytesIO(req.content))
        file.extractall(path= "./shapefiles/" + state_info[1])
        

In [14]:
def open_shape(state):
    '''
    Opens a shapefile as a geopandas object
    inputs:
        state(str): valid values are State FIP codes ("AK", "CA", etc)
    returns:
        geopandas dataframe
    '''
    rv = gpd.read_file('shapefiles/' + dict_of_states[state][1] + '/tl_2010_{}_tabblock10.shp'.format(dict_of_states[state][0]))
    return rv

In [15]:
def create_query(variables, state, county):
    '''
    Creates a query to pull the data from the census API
    Inputs:
        variables(list): list of variables from Census
        state(str): state code (coded with FIPS code, e.g. "01", "02", etc)
        county (str): county code (coded with Census 2010 FIPS code, e.g. "001", "043", etc )
    Output:
        query(str) to call the API
    '''
    query_base = 'https://api.census.gov/data/2010/dec/sf1?get='
    for var in variables:
        query_base = query_base + var +","
    query_base =  query_base + 'NAME&for=block:*&in=state:{}%20county:{}&key=5984786462aaccc39f904bf750cc8274bd40f9f8'.format(state,county)
    return query_base

In [16]:
print(create_query(['P003002','P003003'], '01', '001'))
# s = url.read()
# data = json.loads(s)

https://api.census.gov/data/2010/dec/sf1?get=P003002,P003003,NAME&for=block:*&in=state:01%20county:001&key=5984786462aaccc39f904bf750cc8274bd40f9f8


In [17]:
def create_state_dataframe(state):
    '''
    Creates a dataframe for a selected state
    inputs: 
        state(str): name of the state (e.g. "California")
    returns:
        dataframe
    '''
    ls_dfs = []
    for county in dict_of_STATE_COUNTIES[state][1]:
        print("pulling data from county " + county+ " in the state of " + state)
        query_race_pop = create_query(list(race_and_pop.keys()), dict_of_STATE_COUNTIES[state][0], county)
        query_sex_age = create_query(list(sex_and_age.keys()), dict_of_STATE_COUNTIES[state][0], county)
        response_1 = requests.get(query_race_pop)
        if response_1.status_code==200:
            try:
                data_1 = response_1.json()
                df_1 = pd.DataFrame(data_1[1:], columns=data_1[0]).\
                rename(columns=race_and_pop)
                df_1['GEOID10'] = df_1['state'] + df_1['county'] + df_1['tract'] + df_1['block']
            except:
                print("Error with request")
                pass
        response_2 = requests.get(query_sex_age)
        if response_2.status_code==200:
            try:
                data_2 = response_2.json()
                df_2 = pd.DataFrame(data_2[1:], columns=data_2[0]).\
                rename(columns = sex_and_age)
                df_2['GEOID10'] = df_2['state'] + df_2['county'] + df_2['tract'] + df_2['block']
                df_2 = df_2.drop(columns=['NAME', 'state', 'county', 'tract', 'block'])
                df2 = df_2.set_index('GEOID10')
            except:
                print("Error with request")
                pass        
        merged_dfs = pd.merge(df_1, df_2, on='GEOID10')
        ls_dfs.append(merged_dfs)
    rv = pd.concat(ls_dfs)
    rv = rv[ordered_columns]
    return rv 

In [18]:
def pull_demographics(folder_to_save, save_in_shapes_folder = True):
    '''
    Pulls the demographics variables included in the dictionaries "race_and_pop", and "sex_and _age"
    Saves the datasets as .csv archives into the SAME folders where we saved the shapefiles.
    If you add a different folder to save the files in (i.e. you set a different path in "folder_to_save"),
    you have to set to False the parameter "save_in_shapes_folder"
    
    Inputs:
        folder_to_save(str) - Path where you wanna save your archives. Default is to save each separate csv into
        the folders we created for the shapefiles
        save_in_shapes_folder (boolean): If you want to use a different path, set it to False
    Returns:
        It does not return anything, just saves the data
    '''
    for key, val in dict_of_STATE_COUNTIES.items():
        ls_dfs = []
        for county in val[1]:
            print("pulling data from county " + county + " in the state of " + key)
            query_race_pop = create_query(list(race_and_pop.keys()), dict_of_STATE_COUNTIES[key][0], county)
            query_sex_age = create_query(list(sex_and_age.keys()), dict_of_STATE_COUNTIES[key][0], county)
            response_1 = requests.get(query_race_pop)
            if response_1.status_code == 200:
                try:
                    data_1 = response_1.json()
                    df_1 = pd.DataFrame(data_1[1:], columns=data_1[0]).\
                    rename(columns=race_and_pop)
                    df_1['GEOID10'] = df_1['state'] + df_1['county'] + df_1['tract'] + df_1['block']
                except:
                    print("Error with request")
                    pass
            response_2 = requests.get(query_sex_age)
            if response_2.status_code == 200:
                try:
                    data_2 = response_2.json()
                    df_2 = pd.DataFrame(data_2[1:], columns=data_2[0]).\
                    rename(columns = sex_and_age)
                    df_2['GEOID10'] = df_2['state'] + df_2['county'] + df_2['tract'] + df_2['block']
                    df_2 = df_2.drop(columns=['NAME', 'state', 'county', 'tract', 'block'])
                    df2 = df_2.set_index('GEOID10')
                except:
                    print("Error with request")
                    pass        
            merged_dfs = pd.merge(df_1, df_2, on='GEOID10')
            ls_dfs.append(merged_dfs)
        rv = pd.concat(ls_dfs)
        rv = rv[ordered_columns]
        if save_in_shapes_folder:
            rv.to_csv("./shapefiles/" + key.upper() +"/"+ key +".csv" )
        else:
            rv.to_csv(folder_to_save+ key.upper() +"/"+ key +".csv" )


In [19]:
# This downloads the shapefiles and creates folders for each state
#download_shp_and_save()

In [20]:
pull_demographics("YOUR_PATH", True )

pulling data from county 001 in the state of Alabama
pulling data from county 007 in the state of Alabama
pulling data from county 017 in the state of Alabama
pulling data from county 019 in the state of Alabama
pulling data from county 025 in the state of Alabama
pulling data from county 013 in the state of Alabama
pulling data from county 009 in the state of Alabama
pulling data from county 011 in the state of Alabama
pulling data from county 021 in the state of Alabama
pulling data from county 023 in the state of Alabama
pulling data from county 037 in the state of Alabama
pulling data from county 027 in the state of Alabama
pulling data from county 029 in the state of Alabama
pulling data from county 031 in the state of Alabama
pulling data from county 035 in the state of Alabama
pulling data from county 033 in the state of Alabama
pulling data from county 053 in the state of Alabama
pulling data from county 039 in the state of Alabama
pulling data from county 041 in the state of A

pulling data from county 091 in the state of Arkansas
pulling data from county 071 in the state of Arkansas
pulling data from county 073 in the state of Arkansas
pulling data from county 075 in the state of Arkansas
pulling data from county 077 in the state of Arkansas
pulling data from county 079 in the state of Arkansas
pulling data from county 087 in the state of Arkansas
pulling data from county 089 in the state of Arkansas
pulling data from county 081 in the state of Arkansas
pulling data from county 083 in the state of Arkansas
pulling data from county 085 in the state of Arkansas
pulling data from county 103 in the state of Arkansas
pulling data from county 093 in the state of Arkansas
pulling data from county 095 in the state of Arkansas
pulling data from county 097 in the state of Arkansas
pulling data from county 099 in the state of Arkansas
pulling data from county 101 in the state of Arkansas
pulling data from county 109 in the state of Arkansas
pulling data from county 105

KeyboardInterrupt: 

In [ ]:
# To save the data into a different folder
# pull_demographics("./shapefiles/", False)

## You can merge the shapefiles and csv with the GEOID10 variables

In [21]:
Alabama_DF = create_state_dataframe("Alabama")

pulling data from county 001 in the state of Alabama
pulling data from county 007 in the state of Alabama
pulling data from county 017 in the state of Alabama
pulling data from county 019 in the state of Alabama
pulling data from county 025 in the state of Alabama
pulling data from county 013 in the state of Alabama
pulling data from county 009 in the state of Alabama
pulling data from county 011 in the state of Alabama
pulling data from county 021 in the state of Alabama
pulling data from county 023 in the state of Alabama
pulling data from county 037 in the state of Alabama
pulling data from county 027 in the state of Alabama
pulling data from county 029 in the state of Alabama
pulling data from county 031 in the state of Alabama
pulling data from county 035 in the state of Alabama
pulling data from county 033 in the state of Alabama
pulling data from county 053 in the state of Alabama
pulling data from county 039 in the state of Alabama
pulling data from county 041 in the state of A

In [22]:
Alabama_DF.head()

,NAME,state,county,tract,block,GEOID10,Total population in occupied housing units,Total White alone,Total Black or African American alone,Total American Indian and Alaska Native alone,...,Female 50 to 54 years,Female 55 to 59 years,Female 60 and 61 years,Female 62 to 64 years,Female 65 and 66 years,Female 67 to 69 years,Female 70 to 74 years,Female 75 to 79 years,Female 80 to 84 years,Female 85 years and over
0,"Block 1007, Block Group 1, Census Tract 201, A...",01,001,020100,1007,010010201001007,23,22,1,0,...,3,0,1,2,0,1,0,2,0,0
1,"Block 1010, Block Group 1, Census Tract 201, A...",01,001,020100,1010,010010201001010,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Block 1011, Block Group 1, Census Tract 201, A...",01,001,020100,1011,010010201001011,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Block 1012, Block Group 1, Census Tract 201, A...",01,001,020100,1012,010010201001012,2,2,0,0,...,0,1,0,0,0,0,0,1,0,0
4,"Block 1020, Block Group 1, Census Tract 201, A...",01,001,020100,1020,010010201001020,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
geopandas_AK = open_shape('AK')

In [27]:
geopandas_AK.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,02,013,000100,2132,020130001002132,Block 2132,G5040,R,None,None,S,0,1140917,+56.0119409,-161.0036526,"POLYGON ((-161.00640 56.00869, -161.00668 56.0..."
1,02,013,000100,1015,020130001001015,Block 1015,G5040,R,None,None,S,0,97223,+55.2463629,-162.9248666,"POLYGON ((-162.92847 55.24522, -162.92846 55.2..."
2,02,013,000100,1276,020130001001276,Block 1276,G5040,R,None,None,S,0,75011,+55.2331988,-162.9080366,"POLYGON ((-162.91138 55.23385, -162.91028 55.2..."
3,02,013,000100,2165,020130001002165,Block 2165,G5040,R,None,None,S,0,1315769,+55.7739080,-161.7877456,"POLYGON ((-161.76931 55.78114, -161.77470 55.7..."
4,02,013,000100,1120,020130001001120,Block 1120,G5040,R,None,None,S,0,647915,+54.6743103,-164.4056094,"POLYGON ((-164.39607 54.67081, -164.39744 54.6..."
